In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as opt
import torch.nn.functional as F
from pytorch_metric_learning import losses
from torchvision.io import read_image
from torchvision import transforms
from tqdm.notebook import tqdm
from PIL import Image
import optuna
from optuna.trial import TrialState

In [3]:
random.seed(2020)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
resize_image = transforms.Resize(size=(227, 227))

In [6]:
def calculate_mean(dataset):
    mean = torch.Tensor([0.,0.,0.])
    
    num_samples = len(dataset)
    for i in range(num_samples):
        image, _, _, _ = dataset[i]
        image = image.type(torch.FloatTensor) / 255
        
        for j in range(3):
            mean[j] += torch.mean(image[j, :, :])
            
    mean = mean / num_samples
    return mean

In [7]:
def subtract_mean(image, mean):
    normed_image = torch.zeros(image.size())
    image = image.type(torch.FloatTensor) / 255
    
    for j in range(3):
        normed_image[j] = image[j, :, :] - mean[j]
    return normed_image

In [8]:
class AdienceDataset(Dataset):
    def __init__(self, annot_file, img_dir, train=False, transform=None):
        self.img_lbls = pd.read_csv(annot_file, header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.is_train = train
        self.mean = torch.Tensor([0.,0.,0.])
    
    def __len__(self):
        return len(self.img_lbls)
    
    def __getitem__(self, idx):
        img_file = self.img_lbls.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_file)
        image = read_image(img_path)
#         image = resize_image(image)
        label = self.img_lbls.iloc[idx, 1]
    
        if self.is_train:
            positive_list = self.img_lbls[self.img_lbls.iloc[:, 1] == label].index.values
            positive_list = np.setdiff1d(positive_list, np.array([idx]))
            positive_item = random.choice(positive_list)
            positive_img = self.img_lbls.iloc[positive_item, 0]
            pos_img_path = os.path.join(self.img_dir, positive_img)
            pos_image = read_image(pos_img_path)
            
            negative_list = self.img_lbls[self.img_lbls.iloc[:, 1] != label].index.values
            negative_item = random.choice(negative_list)
            negative_img = self.img_lbls.iloc[negative_item, 0]
            neg_img_path = os.path.join(self.img_dir, negative_img)
            neg_image = read_image(neg_img_path)

            if self.transform:
                image = self.transform(image, self.mean)
                pos_image = self.transform(pos_image, self.mean)
                neg_image = self.transform(neg_image, self.mean)
                
            return image, pos_image, neg_image, label
        else:
            if self.transform:
                image = self.transform(image, self.mean)

            return image, label
    
    def set_mean(self, mean):
        self.mean = mean

In [9]:
train_data = AdienceDataset("../train.csv", "../cropped_Adience/", train=True, transform=subtract_mean)
test_data = AdienceDataset("../test.csv", "../cropped_Adience/", train=False, transform=subtract_mean)
val_data = AdienceDataset("../val.csv", "../cropped_Adience/", train=True, transform=subtract_mean)

In [10]:
# mean = calculate_mean(train_data)
mean = torch.Tensor([0.5062, 0.3581, 0.3099])
mean

tensor([0.5062, 0.3581, 0.3099])

In [11]:
train_data.set_mean(mean)
test_data.set_mean(mean)
val_data.set_mean(mean)

In [12]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor, positive, negative):
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

In [13]:
class CombinedLoss(nn.Module):
    def __init__(self, beta=1.0):
        super(CombinedLoss, self).__init__()
        self.beta = beta
        self.triplet = TripletLoss(margin=1.0)
        self.classification = nn.CrossEntropyLoss()
        
    def forward(self, anchor, positive, negative, classification_out, labels):
        triplet_loss = self.triplet(anchor, positive, negative)
        classification_loss = self.classification(classification_out, labels)
        total_loss = (self.beta * triplet_loss) + classification_loss
        
        return total_loss

In [14]:
class Net(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(num_channels, 96, kernel_size=7, stride=4)
        self.conv11 = nn.Conv2d(96, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv22 = nn.Conv2d(256, 384, kernel_size=1)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        
        self.fc6 = nn.Linear(in_features=13824, out_features=512)
        self.fc7 = nn.Linear(in_features=512, out_features=512)
        self.fc8 = nn.Linear(in_features=512, out_features=num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x, alpha: float):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        sl1 = self.conv11(x)
        sl1 = self.relu(sl1)
        
        x = self.conv2(x)
        # make alpha trainable
        x = self.relu((1-alpha)*x + (alpha*sl1))
        # x = self.relu(x + sl1)
        # x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        sl2 = self.conv22(x)
        sl2 = self.relu(sl2)
        
        x = self.conv3(x)
        x = self.relu((1-alpha)*x + (alpha*sl2))
        # x = self.relu(x + sl2)
        # x = self.relu(x)
        x = self.pool(x)
        
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc7(x)
        x_embeds = x
        x = self.relu(x)
        x = self.dropout(x)
        
        x_classification = self.fc8(x)
        
        return x_embeds, x_classification

In [15]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

In [16]:
accuracy = 0.0
channels = 3  # colour image
classes = 2  # gender classification
embedding_size = 512

In [17]:
best_accu = accuracy
def objective(trial):
    model = Net(channels, classes)
    model.apply(init_weights)
    model = torch.jit.script(model).to(device)
    
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    mom = trial.suggest_float('momentum', 0.7, 0.99)
    wd = trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True)
    optimizer = opt.SGD(model.parameters(), lr=lr, momentum=mom, weight_decay=wd)
    
    alpha = trial.suggest_float("alpha", 0.1, 0.9, step=0.1)
    beta = trial.suggest_float("beta", 0.1, 1.0)
    gamma = trial.suggest_float("gamma", 0.1, 1.0)
    criterion = torch.jit.script(CombinedLoss(beta=beta))
    arc_margin = losses.ArcFaceLoss(classes, embedding_size, margin=0.5, scale=30.0)
    
    batch_size = trial.suggest_int('batch_size', 10, 100)
    num_epochs = 100
    
    print("Learning rate: "+ str(lr))
    print("Momentum: "+ str(mom))
    print("Weight decay: "+ str(wd))
    print("Alpha: "+ str(alpha))
    print("Beta: "+ str(beta))
    print("Gamma: "+ str(gamma))
    print("Batch size: "+ str(batch_size))
    print("Number of epochs: "+ str(num_epochs))
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
        
        # training loop
        running_loss = []
        running_accu = []
        
        model.train()
        for step, (anchor_img, positive_img, negative_img, anchor_label) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
            anchor_img, positive_img, negative_img, anchor_label = anchor_img.to(device), positive_img.to(device), negative_img.to(device), anchor_label.to(device)

            optimizer.zero_grad()
            anchor_out, anchor_pred = model(anchor_img, alpha)
            positive_out, _ = model(positive_img, alpha)
            negative_out, _ = model(negative_img, alpha)
            
            pred = torch.argmax(anchor_pred, 1)
            accuracy = torch.eq(pred, anchor_label).sum() / len(anchor_img)

#             print(anchor_out.shape, positive_out.shape, negative_out.shape, anchor_pred.shape, anchor_label.shape)
            class_triplet_loss = criterion(anchor_out, positive_out, negative_out, anchor_pred, anchor_label)
            arc_loss = arc_margin(anchor_out, anchor_label)
            loss = (gamma * arc_loss) + class_triplet_loss
            loss.backward()
            optimizer.step()

            running_accu.append(accuracy.cpu().detach().numpy())
            running_loss.append(loss.cpu().detach().numpy())
        print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
        
        # validation loop
        val_loss = []
        val_accu = []

        model.eval()
        with torch.no_grad():
            for img, pos_img, neg_img, label in tqdm(val_loader):
                img, pos_img, neg_img, label = img.to(device), pos_img.to(device), neg_img.to(device), label.to(device)
                
                anchor, output = model(img, alpha)
                pos, _ = model(pos_img, alpha)
                neg, _ = model(neg_img, alpha)
                
                pred = torch.argmax(output, 1)
                accuracy = torch.eq(pred, label).sum() / len(img)
                
                class_triplet_loss = criterion(anchor, pos, neg, output, label)
                arc_loss = arc_margin(anchor, label)
                loss = (gamma * arc_loss) + class_triplet_loss
                
                val_accu.append(accuracy.cpu().detach().numpy())
                val_loss.append(loss.cpu().detach().numpy())
        val_accu = np.mean(val_accu)
        val_loss = np.mean(val_loss)
        print("Val Loss: {:.4f} - Val Accuracy: {:.4f}".format(val_loss, val_accu))
        
        trial.report(val_accu, epoch)
        if (epoch >= 20 and val_accu < 0.7) or trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    global best_accu
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model.state_dict(), "../contrastive_arc_model_00.pt")
            
    return val_accu

In [18]:
study = optuna.create_study(direction='maximize',
                            study_name='triplet-arcface-skip-alpha-libfaceid-study',
                            storage='sqlite:///study.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=16)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

# Display the study statistics
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2023-11-07 23:40:56,835] Using an existing study with name 'triplet-arcface-skip-alpha-libfaceid-study' instead of creating a new one.


Learning rate: 0.0011571342051925825
Momentum: 0.908132162898065
Weight decay: 3.2157460854832896e-05
Alpha: 0.8
Beta: 0.5684189289394309
Gamma: 0.7545797353572586
Batch size: 63
Number of epochs: 100


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/222 [00:00<?, ?it/s]

Epoch: 1/100 - Loss: 1.9619 - Accuracy: 0.5183


...

Training:   0%|          | 0/226 [00:00<?, ?it/s]

Epoch: 1/100 - Loss: 1.8274 - Accuracy: 0.4773


  0%|          | 0/29 [00:00<?, ?it/s]

[I 2023-11-08 09:48:57,233] Trial 20 pruned. 


Val Loss: 1.6820 - Val Accuracy: 0.4703

Study statistics: 
  Number of finished trials:  21
  Number of pruned trials:  11
  Number of complete trials:  9


In [19]:
print("Best trial:")
trial = study.best_trial

print("Value: ", trial.value)

print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
Value:  0.9210203289985657
Params: 
    alpha: 0.7000000000000001
    batch_size: 78
    beta: 0.5687545687122332
    gamma: 0.5664070800618352
    learning_rate: 0.00515584900662887
    momentum: 0.9714697358600418
    weight_decay: 1.0248625946806426e-05


In [ ]:
# maximizing accuracy
# base model

Best trial:
Value:  0.9375312924385071
Params: 
    batch_size: 78
    beta: 0.9562296050157876  # triplet loss
    gamma: 0.19011049867353927  # arcface loss
    learning_rate: 0.0014631023046895672
    momentum: 0.9225211100063795
    weight_decay: 0.0069059487633027295

In [ ]:
# skip model

Best trial:
Value:  0.9242949485778809
Params: 
    batch_size: 53
    beta: 0.32523868129239475
    gamma: 0.7236300957389522
    learning_rate: 0.0013057589911738782
    momentum: 0.9300998910920467
    weight_decay: 0.005306632655956046

In [ ]:
# skip model alpha=0.2

Best trial:
Value:  0.9304908514022827
Params: 
    alpha: 0.2
    batch_size: 11
    beta: 0.7122251916319258
    gamma: 0.22543926795028266
    learning_rate: 5.924392960289278e-05
    momentum: 0.9864632282801121
    weight_decay: 0.0004879435121952276

In [ ]:
# skip-alpha 2nd trial

Best trial:
Value:  0.8968684673309326
Params: 
    alpha: 0.7000000000000001
    batch_size: 98
    beta: 0.10000025826802361
    gamma: 0.44700408885364906
    learning_rate: 0.0009127221003519069
    momentum: 0.7088044300089509
    weight_decay: 1.6008833984795333e-05

In [ ]:
# skip-alpha 3rd trial

Best trial:
Value:  0.9210203289985657
Params: 
    alpha: 0.7000000000000001
    batch_size: 78
    beta: 0.5687545687122332
    gamma: 0.5664070800618352
    learning_rate: 0.00515584900662887
    momentum: 0.9714697358600418
    weight_decay: 1.0248625946806426e-05